In [1]:
import argparse
import collections
import datetime
import json
import logging
import operator
import os
import random
import re
import sys
import traceback
from tqdm import tqdm
import io

# from generator_utils import log_statistics, save_cache, query_dbpedia,\
#  strip_brackets, encode, read_template_file
from generator import *
from generator_utils import *

import importlib

from rdflib import URIRef, term, Graph, Literal, Namespace
from rdflib.namespace import OWL,RDF, RDFS, SKOS, XSD

# MAIN

In [2]:
template_file = "../../data/eiopa/1_external/templates.csv"
output_dir = "../../data/eiopa/3_processed"
use_resources_dump = False
file_mode = 'w'

## Initiate Graph 

In [3]:
EIOPA_DATA_PATH = os.path.join("..","..", "data", "external", "eiopa")
GLEIF_DATA_PATH = os.path.join("..","..", "data", "external", "gleif")

g = Graph()

with open(os.path.join(EIOPA_DATA_PATH,'eiopa_register.ttl'), "rb") as fp:
    g.parse(data = fp.read(), format = 'turtle')
# 
with open(os.path.join(GLEIF_DATA_PATH,'gleif-L1-extract.ttl'), "rb") as fp:
    g.parse(data = fp.read(), format = 'turtle')
    
with open(os.path.join(GLEIF_DATA_PATH,'EntityLegalFormData.ttl'), "rb") as fp:
    g.parse(data = fp.read(), format = 'turtle')

print("graph has {} statements.".format(len(g)))

FileNotFoundError: [Errno 2] No such file or directory: '../../data/external/eiopa/eiopa_register.ttl'

## Generate Dataset

In [4]:


def query_database(query):
    """ Returns list of query results """
    results = []
    for row in g.query(query):
        items = []
        for item in row:
            items.append(str(get_name(item)))
        results.append(items)

    return results

In [10]:
templates = read_template_file(template_file)
template = templates[-1]
generator_query = prepare_generator_query(template)
print(generator_query)
results = query_database(generator_query)
for item in results:
    print(item)
    print(build_dataset_pair(item,template))

select distinct ?a where{?s gleif-L1:hasLegalName ?a . ?e gleif-Base:identifies ?s . ?e eiopa-Base:hasOperationStartDate [] . ?e eiopa-Base:hasEUCountryWhereEntityOperates CountryCodes:NL.}
['Blue Square Re N.V.']
{'natural_language': 'When did blue square re n.v. start?', 'query': 'SELECT DISTINCT var_o WHERE  brack_open var_s gleif-L1:hasLegalName blue square re n.v sep_dot sep_dot var_e gleif-Base:identifies var_s sep_dot var_e eiopa-Base:hasOperationStartDate var_o sep_dot brack_close '}
['Mutual Insurance Association "Munis" (Onderlinge Verzekeringsmaatschappij "Munis") U.A.']
{'natural_language': 'When did mutual insurance association "munis"  u.a. start?', 'query': 'SELECT DISTINCT var_o WHERE  brack_open var_s gleif-L1:hasLegalName mutual insurance association "munis"  u.a sep_dot sep_dot var_e gleif-Base:identifies var_s sep_dot var_e eiopa-Base:hasOperationStartDate var_o sep_dot brack_close '}
['N.V. Noordhollandsche van 1816, Levensverzekeringsmaatschappij']
{'natural_langu

In [ ]:
template = templates[100]
print(template.generator_query)
print(template.variables)

In [ ]:
generator_query = prepare_generator_query(template)

In [ ]:
print(generator_query)

In [ ]:
results = query_database(generator_query)

In [ ]:
for item in results:
    print(item)
    print(build_dataset_pair(item,template))